In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [72]:
FeatureSelected = pd.read_csv('/content/drive/My Drive/online_shoppers_intention.csv/data/FeatureSelected', header=None)

FeatureSelected

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419
0,0.12880,0.07283,0.06227,0.05013,0.025640,0.06548,0.12820,0.03715,0.08586,0.06296,0.06316,0.011110,0.044190,0.02579,0.027780,0.05087,0.06517,0.06961,0.17500,0.05882,0.04786,0.08660,0.07407,0.07926,0.06127,0.17080,0.07632,0.12050,0.13970,0.06946,0.10010,0.07911,0.05882,0.023810,0.08449,0.00000,0.07431,0.09749,0.03203,0.12180,...,0.1825,0.21210,0.2422,0.1964,0.19740,0.06575,0.21340,0.24750,0.2173,0.17080,0.18270,0.17390,0.15350,0.19700,0.2280,0.14660,0.15550,0.2625,0.1996,0.17750,0.2060,0.08737,0.14890,0.16630,0.2113,0.15210,0.2264,0.2163,0.20510,0.11200,0.15100,0.16130,0.2475,0.1613,0.2148,0.13620,0.23560,0.2542,0.2216,0.16280
1,99.70000,96.09000,65.13000,84.46000,57.260000,89.88000,99.00000,82.98000,96.08000,83.67000,81.81000,90.240000,59.900000,69.86000,62.060000,65.59000,67.03000,78.00000,65.50000,85.67000,62.56000,89.71000,91.36000,91.88000,84.53000,96.66000,87.78000,91.46000,109.40000,102.90000,104.50000,97.59000,82.14000,68.810000,82.96000,50.41000,84.58000,72.22000,74.22000,85.51000,...,142.0000,142.10000,172.0000,149.6000,138.10000,110.10000,157.10000,125.40000,146.0000,145.30000,113.90000,132.80000,133.30000,145.40000,168.2000,139.80000,152.10000,251.2000,143.6000,122.10000,153.9000,127.30000,143.40000,146.40000,163.2000,106.00000,211.5000,117.7000,113.30000,113.80000,142.20000,160.50000,205.7000,152.0000,160.2000,104.30000,128.70000,179.1000,166.1000,155.00000
2,15.11000,14.50000,10.23000,13.30000,8.964000,13.76000,15.15000,12.98000,14.67000,13.10000,12.84000,14.230000,9.507000,11.02000,9.565000,10.01000,10.41000,12.33000,10.31000,13.46000,9.73300,14.11000,14.08000,14.24000,13.11000,15.53000,13.67000,13.83000,16.34000,16.11000,16.36000,15.10000,12.83000,10.920000,13.06000,7.93000,13.34000,10.76000,11.54000,13.14000,...,20.8200,21.20000,26.2300,20.8000,21.08000,17.26000,23.17000,16.35000,22.0300,22.66000,17.04000,20.38000,20.38000,21.53000,25.6800,21.44000,22.96000,36.0400,21.5700,17.39000,23.9600,19.18000,21.53000,22.75000,23.8600,16.01000,30.7900,17.1100,16.41000,17.58000,21.86000,23.73000,29.9200,23.2300,24.3000,15.29000,17.52000,24.2900,25.4500,23.69000
3,0.04781,0.03110,0.02076,0.02923,0.005917,0.02749,0.03384,0.01115,0.01723,0.01349,0.02278,0.004167,0.007799,0.01290,0.009259,0.02180,0.01504,0.02230,0.04375,0.02864,0.02872,0.02272,0.06556,0.02315,0.02233,0.06987,0.01863,0.03745,0.07064,0.02957,0.02648,0.02780,0.02941,0.007937,0.03350,0.00000,0.01770,0.03029,0.01201,0.03485,...,0.1021,0.09113,0.1562,0.1198,0.08824,0.02739,0.08481,0.09711,0.1259,0.08744,0.06463,0.05736,0.06254,0.08293,0.1155,0.09063,0.06431,0.1689,0.1002,0.09194,0.1194,0.02307,0.07762,0.09183,0.1121,0.06759,0.1003,0.0852,0.08172,0.04335,0.09451,0.09431,0.1471,0.1015,0.1322,0.06139,0.09429,0.1474,0.1389,0.09791
4,87.46000,85.63000,60.34000,82.61000,51.710000,78.04000,86.91000,74.72000,87.21000,75.71000,73.34000,82.610000,54.340000,64.55000,54.660000,59.20000,60.11000,71.80000,58.79000,81.37000,58.79000,79.19000,87.91000,83.19000,73.81000,86.49000,76.95000,79.78000,95.77000,94.57000,84.74000,86.60000,77.79000,62.110000,74.34000,43.79000,77.22000,63.95000,67.41000,75.17000,...,128.9000,102.40000,144.4000,124.4000,117.80000,96.71000,102.90000,102.10000,133.8000,123.70000,91.12000,112.40000,117.50000,117.30000,138.1000,127.70000,111.20000,186.9000,119.0000,109.80000,128.1000,107.10000,118.40000,121.30000,137.8000,92.33000,152.1000,103.8000,88.64000,97.26000,120.80000,130.50000,165.5000,134.7000,137.8000,93.77000,103.40000,143.0000,142.0000,131.20000
5,13.54000,13.08000,9.50400,13.03000,8.196000,12.05000,13.49000,11.76000,13.64000,11.94000,11.52000,13.050000,8.618000,10.17000,8.598000,9.17300,9.46500,11.31000,9.02900,12.78000,8.88800,12.31000,13.53000,12.86000,11.45000,13.34000,12.000

In [73]:
NoOfData   = int(FeatureSelected.shape[1]/2) 
Fold       = 5       

FeatNo     = int(FeatureSelected.shape[0]) 
FoldDataNo = int(NoOfData/Fold)          

print(FeatNo)
print(FoldDataNo)

14
42


In [74]:
benign_set   = np.array(FeatureSelected.iloc[: , :NoOfData])
malignant_set = np.array(FeatureSelected.iloc[: , NoOfData:])
FeatureSelected_Reshaped = pd.DataFrame(np.concatenate([benign_set , malignant_set] , axis=0))
FeatureSelected_Reshaped.shape

(28, 210)

In [75]:
for i in range(Fold):
    
    temp_Valid_benign   = FeatureSelected_Reshaped.iloc[:FeatNo , FoldDataNo*i : FoldDataNo*(i+1)]
    temp_Valid_malignant = FeatureSelected_Reshaped.iloc[FeatNo: , FoldDataNo*i : FoldDataNo*(i+1)]
    temp_Valid = pd.DataFrame(np.transpose(np.concatenate([temp_Valid_benign, temp_Valid_malignant] , axis=1)))
    
    s = 'Validation_Fold%d = temp_Valid'%(i+1)
    exec(s)

    
for i in range(Fold):
    
    temp_Train_Front = FeatureSelected_Reshaped.iloc[:,:FoldDataNo*i]
    temp_Train_Back  = FeatureSelected_Reshaped.iloc[:,FoldDataNo*(i+1):]
    temp_Train_Total = np.concatenate([temp_Train_Front , temp_Train_Back] , axis=1)
    temp_Train_Final = pd.DataFrame(np.transpose(np.concatenate([temp_Train_Total[:FeatNo,:],temp_Train_Total[FeatNo:,:]] , axis=1)))
    
    s ='Training_Fold%d  = temp_Train_Final'%(i+1)
    exec(s)

In [76]:
Validation_Fold1.shape

(84, 14)

In [77]:
Training_Fold1.shape

(336, 14)

In [78]:
NoOfLabel_Train = int(Training_Fold1.shape[0]/2)
NoOfLabel_Valid = int(Validation_Fold1.shape[0]/2)

print(NoOfLabel_Train)
print(NoOfLabel_Valid)

168
42


In [79]:
TrainingFold_Label   = np.zeros(2*NoOfLabel_Train , dtype=int)
ValidationFold_Label = np.zeros(2*NoOfLabel_Valid , dtype=int)

TrainingFold_Label[NoOfLabel_Train:] = 1

ValidationFold_Label[NoOfLabel_Valid:] = 1

TrainingFold_Label   = pd.Series(TrainingFold_Label)
ValidationFold_Label = pd.Series(ValidationFold_Label)


ValidationFold_Label

0     0
1     0
2     0
3     0
4     0
     ..
79    1
80    1
81    1
82    1
83    1
Length: 84, dtype: int64

In [80]:
TrainingFold_Label_forANN   = np.zeros((NoOfLabel_Train*2,2) , dtype=int)
ValidationFold_Label_forANN = np.zeros((NoOfLabel_Valid*2,2) , dtype=int)

TrainingFold_Label_forANN[:NoOfLabel_Train , 0] = 1   
TrainingFold_Label_forANN[NoOfLabel_Train: , 1] = 1  

ValidationFold_Label_forANN[:NoOfLabel_Valid , 0] = 1  
ValidationFold_Label_forANN[NoOfLabel_Valid: , 1] = 1 

TrainingFold_Label_forANN   = pd.DataFrame(TrainingFold_Label_forANN)
ValidationFold_Label_forANN = pd.DataFrame(ValidationFold_Label_forANN)

#ValidationFold_Label_forANN
TrainingFold_Label_forANN

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
331,0,1
332,0,1
333,0,1
334,0,1


In [81]:
for i in range(Fold): 
    
    path1 = '/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Training_Fold%d'  %(i+1)
    path2 = '/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Validation_Fold%d'%(i+1)
    
    c1 = 'Training_Fold%d.to_csv(  path1, sep = ",", header = None, index = None)'%(i+1)
    c2 = 'Validation_Fold%d.to_csv(path2, sep = ",", header = None, index = None)'%(i+1)
    exec(c1)
    exec(c2)

In [82]:
test = pd.read_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/ValidationFold_Label_forANN', header=None)

test.shape

(84, 2)

In [83]:
ValidationFold_Label_forANN


,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
79,0,1
80,0,1
81,0,1
82,0,1


In [84]:
TrainingFold_Label.to_csv(  '/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/TrainingFold_Label', header = None, index = None)
ValidationFold_Label.to_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/ValidationFold_Label', header = None, index = None)

TrainingFold_Label_forANN.to_csv(  '/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/TrainingFold_Label_forANN', sep = ",", header = None, index = None)
ValidationFold_Label_forANN.to_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/ValidationFold_Label_forANN', sep = ",", header = None, index = None)

In [85]:
Training_All = np.transpose(FeatureSelected)

Training_All.shape

(420, 14)

In [86]:
Training_All_Label = np.zeros(NoOfData*2)

Training_All_Label[NoOfData:] = 1    # 고장데이터(학습용) Label 값 = 1
Training_All_Label = pd.Series(Training_All_Label)

Training_All_Label.shape

(420,)

In [87]:
Training_All_Label_forANN = np.zeros((NoOfData*2,2))

Training_All_Label_forANN[:NoOfData , 0] = 1
Training_All_Label_forANN[NoOfData: , 1] = 1
Training_All_Label_forANN = pd.DataFrame(Training_All_Label_forANN)

Training_All_Label_forANN

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
415,0.0,1.0
416,0.0,1.0
417,0.0,1.0
418,0.0,1.0


In [88]:
Training_All.to_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Training_All', sep = ",", header = None, index = None)
Training_All_Label.to_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Training_All_Label', sep = ",", header = None, index = None)
Training_All_Label_forANN.to_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Training_All_Label_forANN', sep = ",", header = None, index = None)